## Import libraries

In [1]:
# Notebook setup
import pickle
import os
import datetime
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
sess = InteractiveSession(config=config)

plt.rcParams['figure.figsize'] = [14, 7]
os.environ['PYTHONHASHSEED'] = '0'

# Set random seed to create reproducable results
the_seed = 1234
np.random.seed(the_seed)
random.seed(the_seed)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(the_seed)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [2]:
import find

## Settings
We use the biosbias setting as an example to show how to load and run a pre-trained model.

In [3]:
EMBEDDING_DIM = 300
EMBEDDING_PATH = f"GLoVe/glove.6B.{EMBEDDING_DIM}d.txt" # Path to your glove embeddings

In [4]:
DATA_PATH = 'EMNLP2020-Data2Share/Data2Share/'
MAIN_DATASET = 'YelpSmall500'
SECOND_DATASET = None
THIRD_DATASET = None
GENDER_BIAS = False

### !!! Please change MODEL_FILE_PATH to the path of a pre-trained model you want to run in your local computer

In [5]:
MODEL_FILE_PATH = 'Yelp-AmazonProducts-CNNs/1A/YelpSmall500_CNN_20200515014923.h5'
MODEL_ARCH = 'CNN'
MAXLEN = 150
FILTERS = [(10, 2), (10, 3), (10, 4)] # Ten filters of each window size [2,3,4]
BATCH_SIZE = 128

## Prepare embeddings and dataset

In [6]:
# 0. Load GloVe embeddings
embedding_matrix, vocab_size, index2word, word2index = find.get_embedding_matrix(EMBEDDING_PATH, EMBEDDING_DIM, pad_initialisation = "zeros")

Loading Glove Model


400000it [00:27, 14524.29it/s]


Done. 400000  words loaded!


In [7]:
# 1. Load datasets and prepare inputs
# 1.1 Main dataset
data_1 = pickle.load(open(DATA_PATH + f'all_data_{MAIN_DATASET}.pickle', 'rb'))
class_names = data_1['class_names']
X_train_1, X_validate_1, X_test_1 = find.get_data_matrix(data_1['text_train'], word2index, MAXLEN), \
                                    find.get_data_matrix(data_1['text_validate'], word2index, MAXLEN), \
                                    find.get_data_matrix(data_1['text_test'], word2index, MAXLEN)
y_test_1 = data_1['y_test']
#gender_test_1 = data_1['gender_test'] if GENDER_BIAS else None

# 1.2 Second dataset
if SECOND_DATASET is not None:
    data_2 = pickle.load(open(DATA_PATH + f'all_data_{SECOND_DATASET}.pickle', 'rb'))
    X_test_2, y_test_2 = find.get_data_matrix(data_2['text_test'], word2index, MAXLEN), data_2['y_test']
    gender_test_2 = data_2['gender_test'] if GENDER_BIAS else None
else:
    X_test_2, y_test_2, gender_test_2 = None, None, None

# 1.3 Third dataset
if THIRD_DATASET is not None:
    data_3 = pickle.load(open(DATA_PATH + f'all_data_{THIRD_DATASET}.pickle', 'rb'))
    X_test_3, y_test_3 = find.get_data_matrix(data_3['text_test'], word2index, MAXLEN), data_3['y_test']
    gender_test_3 = data_3['gender_test'] if GENDER_BIAS else None
else:
    X_test_3, y_test_3, gender_test_2  = None, None, None

100%|██████████| 38000/38000 [00:15<00:00, 2516.60it/s]


## Create, load, and run the model

In [8]:
# 3. Create a model
if MODEL_ARCH == 'CNN':
    model = find.get_CNN_model(vocab_size, EMBEDDING_DIM, embedding_matrix, MAXLEN, class_names, FILTERS)
else:
    assert False, f"Unsupported model architecture: {MODEL_ARCH}"










__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     120000600   input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 149, 10)      6010        embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 148, 10)      9010        embedding_1[0][0]                
__________________________________________________________________________________________________
c

In [9]:
# 4. Load pre-trained model weights
model.load_weights(MODEL_FILE_PATH)

In [10]:
# 5. Evaluate the model
if not GENDER_BIAS:
    find.evaluate_all(model, class_names, BATCH_SIZE, X_test_1, y_test_1, X_test_2, y_test_2, X_test_3, y_test_3, result_path = None, model_name = 'original')
else:
    find.evaluate_all(model, class_names, BATCH_SIZE, X_test_1, y_test_1, X_test_2, y_test_2, gender_test_2, result_path = None, model_name = 'original')

Evaluate with the original test set:

{'per_class': {0: {'all_positive': 25161,
                   'all_true': 19000,
                   'class_f1': 0.7917846063268495,
                   'class_name': 'Negative',
                   'class_precision': 0.6948451969317595,
                   'class_recall': 0.9201578947368421,
                   'true_positive': 17483},
               1: {'all_positive': 12839,
                   'all_true': 19000,
                   'class_f1': 0.71120324130783,
                   'class_name': 'Positive',
                   'class_precision': 0.8818443804034583,
                   'class_recall': 0.5958947368421053,
                   'true_positive': 11322}},
 'total': {'accuracy': 0.7580263157894737,
           'macro_f1': 0.7728883370920798,
           'macro_precision': 0.7883447886676089,
           'macro_recall': 0.7580263157894737,
           'micro_f1': 0.7580263157894737,
           'micro_precision': 0.7580263157894737,
           'micro_rec